In [ ]:
#1. Environment Setup
################################################################################
#01) Setting Up Google Colab : 런타임 > 런타임유형변경 > GPU&표준 
#02) Checking GPU Version
#03) Link Google Colab to Google Drive
import os
import sys
from google.colab import drive
drive.mount('/content/gdrive')
# 에러시 런타임 초기화 
if not os.path.exists('/content/gdrive/MyDrive/Class/dl082/YOLOV4/'):
  !mkdir -p '/content/gdrive/MyDrive/Class/dl082/YOLOV4/'
%cd '/content/gdrive/MyDrive/Class/dl082/YOLOV4/'
# 런타임 상태 확인
!nvidia-smi
# Cuda Version
!/usr/local/cuda/bin/nvcc --version
# ls
!ls -lia;pwd

In [ ]:
#04) Clone the repository using the below command.
####
import tensorflow as tf
print(tf.__version__)

%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4
!rm -fr darknet/
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
#05) Some changes in configuration
####
# In order to make use of YOLO most efficiently, we make some necessary changes to the ‘make’ configuration file. 
# Basically, we are making sure that OpenCV is installed with CUDA and GPU support in order to make computations faster.
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENMP=0/OPENMP=1/' Makefile

In [ ]:
#06) Building Darknet
####
# Next, we build the darknet with the below command –
!make

In [ ]:
#07) inference on an image by an pre-trained model yolov4
####
# YOLOV4: Backbone(CSPDarknet53), Neck(SPP,PAN), Head(YOLOv3)
# https://arxiv.org/abs/2004.10934
##
# cfg-file: 
# https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.cfg
# weights-file: 
# https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
##
if not os.path.exists('/content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/weights'):
  !mkdir -p '/content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/weights'
%cd '/content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/weights'
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
#2. Training Custom YOLOv4 Object Detection Model
################################################################################
#01) Preparing Custom dataset
###
#(1) Download the Labeled Custom Dataset
#(2) Custom cfg file
#(3) obj.data and obj.names files
#(4) process.py file (to create train.txt and test.txt files for training)
####

In [ ]:
#(1) roboflow.com에 image를 upload하여 레이블 작업을 한다. 
# 그리고 jpg와 txt파일을 다운받는다.

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="zoBg2GLDUtBZCo80f9KE")
project = rf.workspace("class-kybcx").project("car-gnlqb")


if not os.path.exists('/content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images'):
  !mkdir -p /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images

dataset = project.version(1).download("darknet")
!mv Car-1 Car

%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images/Car/
!mv /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images/Car/test/* .
!mv /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images/Car/valid/* .
!mv /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images/Car/train/* .
!rm -fr test train valid
!ls -lia;pwd

In [ ]:
#(2) Create your data/train.txt and data/test.txt files 

import glob, os
# Current directory
#current_dir = os.path.dirname(os.path.abspath(__file__))
#print(current_dir)
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/
images_dir = '/content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/images/Car'
print(images_dir)

# Percentage of images to be used for the test set
percentage_test = 10;

# Create and/or truncate train.txt and test.txt
file_train = open('data/train.txt', 'w')
file_test = open('data/test.txt', 'w')

# Populate train.txt and test.txt
counter = 1
index_test = round(100 / percentage_test)
for pathAndFilename in glob.iglob(os.path.join(images_dir, "*.jpg")):
    print(pathAndFilename)
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:
        counter = 1
        file_test.write("images" + "/" + title + '.jpg' + "\n")
    else:
        file_train.write("images" + "/" + title + '.jpg' + "\n")
        counter = counter + 1

In [ ]:
#(3) cfg/yolov4-custom.cfg을 data/yolov4-BCCD.cfg로 복사합니다.
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/
!cp cfg/yolov4-custom.cfg yolov4-Car.cfg

# open the yolov4-BCCD.cfg file and change the next
##
# line 06: batch=32 # [updated]
# line 08: width=416 # 416 [updated]
# line 09: height=416 # 416 [updated]
# line 20: max_batches=6000 # (No of classes) * 2000
# line 22: steps=4800,5400 # 0.8*max_batches, 0.9*max_batches [updated]
# lines 970,1058,1146 : classes=3 # 2 [updated]
# lines 963,1051,1139 : filters=24 # (classes+5)*3 [updated] 
# - in the convolutional layer immediately before each 3 yolo layers
####

In [ ]:
#(4) Create your BCCD.data and BCCD.names files into data folder
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet/
!cp cfg/coco.data cfg/Car.data
!cp cfg/coco.names cfg/Car.names
####
#BCCD.data
# classes = 3
# train  = data/train.txt
# valid  = data/test.txt
# names = cfg/BCCD.names
# backup = backup/
####
#BCCD.names
# Platelets
# RBC
# WBC

In [ ]:
#2. Training Custom YOLOv4 Object Detection Model
################################################################################
#01) Training 
####
# We first need to understand the following command-line syntax and the parameters for training the custom YOLOv4 model with darknet.
#!./darknet detector train <path to obj.data> <path to custom config> <path to weight file> -dont_show -map
# <path to obj.data>: Path to obj.data.
# <path to custom config>: Path to obj.cfg file.
# <path to weight file>: Path to the downloaded(yolov4.conv.137) weight file
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet
!chmod +x darknet
!./darknet detector train \
    cfg/Car.data \
    yolov4-Car.cfg \
    weights/yolov4.conv.137 -dont_show -map

####
#  Last accuracy mAP@0.50 = 85.30 %, best = 87.86 % 
# 6000: 4.449986, 4.189793 avg loss, 0.000010 rate, 3.793319 seconds, 192000 images, 0.076887 hours left
# Resizing to initial size: 416 x 416  try to allocate additional workspace_size = 52.43 MB 
# CUDA allocate done! 
####

In [ ]:
#3. Testing Our Model
################################################################################
#01) Checking the mAP
####
# We can see it’s around 77 percent which is a pretty good score.
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet

!./darknet detector map \
  cfg/Car.data \
  yolov4-Car.cfg \
  backup/yolov4-Car_last.weights

In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


In [ ]:
#02) Testing Our Custom YOLOv4 Model
####
# Test Command Format
# !./darknet detector test <path to obj.data> <path to cfg file> <weight file> <query image/video> -thresh <x>
# <path to obj.data>: Path to obj.data.
# <path to custom config>: Path to obj.cfg file.
# <path to weight file>: Path to the weight file you want to run detections with
# <query image>: Path to the image you want to run detections on.
# <x>: Minimum confidence value i.e. threshold.

# Let us run our custom YOLOv4 model on some images and see the results visually.
%cd /content/gdrive/MyDrive/Class/dl082/YOLOV4/darknet
!chmod +x darknet
!./darknet detect \
  yolov4-Car.cfg \
  backup/yolov4-Car_last.weights \
  ../workspace/Car/KakaoTalk_20220406_190817990_04.jpg \
  -thresh 0.3
imShow('predictions.jpg')